# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [153]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [154]:
cities = pd.read_csv('../output_data/cities.csv')
#cities.set_index(["City"],inplace=True)
cities


,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05
...,...,...,...,...,...,...,...,...,...,...
543,543,yabrud,0,SY,1558378840,32,33.97,36.66,80.60,8.05
544,544,paraiso,5,MX,1558378898,4,24.01,-104.61,84.20,16.11
545,545,veraval,0,FR,1558378898,62,49.65,0.71,61.00,8.05
546,546,novyy urgal,100,RU,1558378899,93,51.07,132.56,49.74,2.68


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [155]:
gmaps.configure(api_key=g_key)
locations= cities[["Lat","Lng"]]
humidity = cities["Humidity"]

locations

,Lat,Lng
0,68.05,39.51
1,-20.41,57.70
2,77.48,-69.36
3,40.71,112.04
4,-33.93,18.42
...,...,...
543,33.97,36.66
544,24.01,-104.61
545,49.65,0.71
546,51.07,132.56


In [156]:
# Create a poverty Heatmap layer
fig = gmaps.figure(center=(68.0, 40.0), zoom_level=1.8)

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100
                                 ,point_radius = 1)
                                 

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [148]:
#A max temperature lower than 80 degrees but higher than 70.
#Wind speed less than 10 mph.
#Zero cloudiness.
#Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

cities_w = cities

cities_w.drop(cities_w[cities_w["Max Temp"]>80 ].index,inplace=True)
cities_w.drop(cities_w[cities_w["Max Temp"]<70 ].index,inplace=True)
cities_w.drop(cities_w[cities_w["Wind Speed"]>=10 ].index,inplace=True)
cities_w.drop(cities_w[cities_w["Cloudiness"]!= 0 ].index,inplace=True)
cities_w


,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
7,7,opuwo,0,NaN,1558378523,23,-18.06,13.84,72.06,5.44
70,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32
88,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50
138,138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06
139,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59
176,176,nador,0,MA,1558378499,40,35.17,-2.93,75.20,6.93
297,297,mogok,0,MM,1558378844,51,22.92,96.51,72.24,2.77
444,444,birjand,0,IR,1558378880,26,32.86,59.22,71.60,9.17


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [150]:
# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json" 
hotel_df= []
hotel_lat=[]
hotel_lng=[]
for index, row in cities_w.iterrows():
    
    target_coordinates = f"{row['Lat']}, {row['Lng']}"
    target_search = "hotel"
    target_radius = 5000
    target_type = "hotel"
    params = {
    "location": target_coordinates,
    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
    }
    response = requests.get(base_url, params=params)
    # convert response to json
    places_data = response.json()

    # Print the json (pretty printed)
    #print(json.dumps(places_data, indent=4, sort_keys=True))
    print(places_data["results"][0]["name"])
    print(places_data["results"][0]["vicinity"])
    hotel_df.append(places_data["results"][0]["name"])
    hotel_lat.append(places_data["results"][0]["geometry"]["location"]["lat"])
    hotel_lng.append(places_data["results"][0]["geometry"]["location"]["lng"])
cities_w["Hotel Name"]=hotel_df
cities_w["hlat"]=hotel_lat
cities_w["hlng"]=hotel_lng
cities_w
    
   

Opuwo Country Lodge
Opuwo
Escarpas Jatobá Hotel
Praça Dr. Marcos Valle Mendes, 46, Capitólio 37930-000, Capitólio
Pond Country Club
Estrada RS 784 KM 7 - Lagoa Country Club, Cidreira
Palissandre Cote Ouest resort & SPA
Morondava
Dandeli Lake County
Dandeli, Jagalbet
Hotel Marchica Lagoon Resort, Nador Morocco
Cité d'Atalayoun
Mount Mogok Hotel
Mogok
Moghaddam Hotel
Birjand, جنب پارک آزادی، بلوار جمهوری


,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name,hlat,hlng
7,7,opuwo,0,NaN,1558378523,23,-18.06,13.84,72.06,5.44,Opuwo Country Lodge,-18.044682,13.833500
70,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32,Escarpas Jatobá Hotel,-20.644364,-46.010648
88,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50,Pond Country Club,-30.167805,-50.255742
138,138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06,Palissandre Cote Ouest resort & SPA,-20.300026,44.267937
139,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59,Dandeli Lake County,15.323247,74.506850
176,176,nador,0,MA,1558378499,40,35.17,-2.93,75.20,6.93,"Hotel Marchica Lagoon Resort, Nador Morocco",35.206750,-2.912102
297,297,mogok,0,MM,1558378844,51,22.92,96.51,72.24,2.77,Mount Mogok Hotel,22.909694,96.503862
444,444,birjand,0,IR,1558378880,26,32.86,59.22,71.60,9.17,Moghaddam Hotel,32.886115,59.220424


In [191]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in cities_w.iterrows()]
locations = cities_w[["hlat", "hlng"]].astype(float)

In [193]:

[f"City: {row['City']},Country:{row['Country']}, Hotel:{row['Hotel Name']} " for index,row in cities_w.iterrows()]

['City: opuwo,Country:nan, Hotel:Opuwo Country Lodge ',
 'City: ponta do sol,Country:BR, Hotel:Kanto Island Inn Suites & Spa ',
 'City: cidreira,Country:BR, Hotel:Pond Country Club ',
 'City: morondava,Country:MG, Hotel:Kimony Resort Hotel ',
 'City: vaini,Country:IN, Hotel:Dandeli Lake County ',
 'City: nador,Country:MA, Hotel:Hotel Marchica Lagoon Resort, Nador Morocco ',
 'City: mogok,Country:MM, Hotel:Mount Mogok Hotel ',
 'City: birjand,Country:IR, Hotel:Sepehr Hotel ']

In [166]:
# Add marker layer ontop of heat map


hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=5,
    info_box_content=[f"City: {row['City']},Country:{row['Countr']}, Hotel:{row['Hotel Name']} " for index,row in cities_w.iterrows()]
)


fig = gmaps.figure(center=(68.0, 40.0), zoom_level=1.8)
fig.add_layer(hotel_layer)
fig.add_layer(heat_layer)

fig


Figure(layout=FigureLayout(height='420px'))